In [1]:
import networkx as nx
import netwulf as nw
import pandas as pd
import numpy as np
import json
from collections import defaultdict
from copy import deepcopy

In [2]:
courses = pd.read_csv('../data/UpdatedCourses.csv', index_col=0)

In [3]:
department_list = sorted(list(set(courses.Institut)))

colour_list = ['#E74C3C', '#8E44AD', '#3498DB', '#2ECC71', '#F39C12', '#F1C40F', '#F5B7B1', '#5D6D7E', 
               '#AED6F1','#F5B7B1', '#FCF3CF', '#DCB9ED', '#8F2323', '#8F6A23', '#4F8F23', '#23628F', 
               '#6B238F', '#AED6F1','#A3E4D7']

department_to_colour_dict = {department: colour for department, colour in zip(department_list, colour_list)}

In [4]:
all_ids = set(courses['Kursus id'])

In [5]:
SPECIAL_CASE_COURSES = {'02312', '02314', '62531', '62532'}

In [6]:
class Course:

    def __init__(self, series):
        self.course_id = series['Kursus id']
        self.danish_title = self.trim_title(series['Dansk titel'])
        self.english_title = self.trim_title(series['Engelsk titel'])
        self.points = series['Point( ECTS )']
        self.course_type = set(series['Kursustype'].split(','))
        self.location = series['Undervisningens placering']
        if series['Anbefalede forudsætninger'] is not np.nan:
            self.prerequisites = list(ele for ele in set(series['Anbefalede forudsætninger'].split(',')) if not ele == self.course_id)
        else:
            self.prerequisites = []
        self.ins = []
        self.outs = []
        self.responsible = series['Kursusansvarlig']
        self.department = series['Institut']
        self.link = series['Link']
        self.programmes = {program: kind for program, kind in series[10:].items()}
        self.level = None if not self.course_id in SPECIAL_CASE_COURSES else 0

    def trim_title(self, title, k=16):
        ki = 0
        trimmed_title = ""
        for i, letter in enumerate(title):
            if letter == " " and i-ki > k:
                print(title[ki:i], ki)
                trimmed_title += title[ki:i] + "\n"
                ki = i+1
        trimmed_title += title[ki:]
        return trimmed_title

    def to_dict(self):
        course_dict = {
            'ins': [c.course_id for c in self.ins],
            'outs': [c.course_id for c in self.outs],
            'danish title': self.danish_title,
            'english title': self.english_title,
            'points': self.points,
            'course type': list(self.course_type),
            'location': self.location,
            'responsible': self.responsible,
            'department': self.department,
            'link': self.link,
            #'programmes': self.programmes,
            'level': self.level
        }
        return course_dict

In [7]:
class CourseList:

    def __init__(self, catalogue={}, courses=set(), N=0):

        if isinstance(catalogue, pd.DataFrame):
            self.catalogue = {}
            self.courses = set()
            self.N = 0
            self.create_course_list_from_dict(catalogue)
        else:
            self.catalogue = catalogue
            self.courses = courses
            self.N = N
            assert len(courses) == N

    def add_course(self, series):
        if series['Kursus id'] not in self.courses:
            self.N += 1
            self.courses.add(series['Kursus id'])
        self.catalogue[series['Kursus id']] = Course(series)

    def add_course_connections(self):
        for course in self.catalogue.values():
            if course.prerequisites is not None and course.prerequisites != 'Unknown':
                for prerequisite in course.prerequisites:
                    if prerequisite in self.courses:
                        course.ins.append(self.catalogue[prerequisite])
                        self.catalogue[prerequisite].outs.append(course)

    def create_course_list_from_dict(self, courses_df):
        for i in range(len(courses_df)):
            self.add_course(courses_df.loc[i])
        self.add_course_connections()

    def lookup(self, course_id):
        return self.catalogue[course_id]

    def assign_levels(self):
        for course in self.catalogue.values():
            #print(course.course_id, course.danish_title)
            self._assign_levels(course, None)

    def _assign_levels(self, course, prev):
        if course.level is not None:
            #print('\t   Already visited')
            return

        loop = False
        max_level = 0
        for prerequisite_course in course.ins:
            if prerequisite_course.course_id == prev:
                loop = True
                continue
            #print('\t', prerequisite_course.course_id, prerequisite_course.danish_title)
            if not self._assign_levels(prerequisite_course, course.course_id):
                max_level = max((max_level, prerequisite_course.level + 1))
        course.level = max_level

        if loop:
            return True
    
    def reset_levels(self):
        for course in self.catalogue.values():
            if not course.course_id in SPECIAL_CASE_COURSES:
                course.level = None

    def to_dict(self):
        network_dict = {}
        for course_id, course in self.catalogue.items():
            network_dict[course_id] = course.to_dict()
        return network_dict
    
    def select_by_department(self, department):
        department_catalogue = {}
        courses = set()
        for course_id, course in self.catalogue.items():
            if course.department == department:
                department_catalogue[course_id] = deepcopy(course)
                courses.add(course.course_id)
                for prereq in course.ins:
                    self._select_prerequisites(prereq, department_catalogue, courses)

        # Remove courses in "outs" which are no longer part of the graph
        for course in department_catalogue.values():
            safe_outs = []
            for i, connection in enumerate(course.outs):
                if connection.course_id in courses:
                    safe_outs.append(connection)
            course.outs = safe_outs

        return CourseList(department_catalogue, courses, len(courses))
    
    def select_by_program(self, program):
        raise('Not implemented yet')
    
    def _select_prerequisites(self, course, catalogue, courses):
        if course.course_id in courses:
            return
        catalogue[course.course_id] = deepcopy(course)
        courses.add(course.course_id)
        for prereq in course.ins:
            self._select_prerequisites(prereq, catalogue, courses)
    
    def graph(self, danish=False):
        
        G = nx.DiGraph()

        # Add nodes
        for course in self.catalogue.values():
            #print(courses['Point( ECTS )'][i])
            G.add_node(
                course.course_id,
                name = course.danish_title if danish else course.english_title,
                department = course.department,
                size = max(course.points, 1),
                responsible = course.responsible,
                group=department_to_colour_dict[course.department]
            )

        # Add edges
        for course in self.catalogue.values():    
            for prereq in course.ins:
                G.add_edge(prereq.course_id, course.course_id)
        
        return G

In [8]:
cl = CourseList(courses)
cl.assign_levels()

Repetitionskursus 0
Advanced Engineering 0
Advanced Engineering 0
Advanced Engineering 0
Advanced Engineering 0
Advanced Engineering 0
Diskret matematik 0
Discrete mathematics 0
Videregående Lineær 0
Matematik 2 for Matematik 0
Advanced Mathematics 0
2 for Mathematics 21
Advanced Engineering 0
Advanced Engineering 0
Advanced Engineering 0
Topologiske grundbegreber 0
Fundamental topological 0
concepts and metric 24
Differentialgeometri 0
Differential geometry 0
Videregående modellering 0
Advanced Modelling 0
Funktionsrum og matematisk 0
Function spaces and 0
Algebraiske Fejlrettende 0
Algebraic Error-Correcting 0
Computational Discrete 0
Computational Discrete 0
Introduktion til partielle 0
Introduction to Partial 0
Introduktion til Dynamiske 0
Introduction to Dynamical 0
Videregående dynamiske 0
Advanced Dynamical 0
Videregående dynamiske 0
Advanced Dynamical 0
Systems: Applications 19
Fagprojekt - Bachelor 0
Project work - Bachelor 0
of Mathematics and 24
Videregående emner 0
Advanced

In [9]:
print(cl.catalogue['02465'].danish_title)

Introduktion til reinforcement
learning og kontrol


In [10]:
G = cl.graph()
network_G, config_G = nw.visualize(G, plot_in_cell_below=False)
data_G = cl.to_dict()

In [11]:
with open(f'../networks/network.json', 'w') as fp:
    json.dump(network_G, fp)
with open(f'../networks/config.json', 'w') as fp:
    json.dump(config_G, fp)
with open(f'../networks/data.json', 'w') as fp:
    json.dump(data_G, fp)

In [15]:
cl2 = cl.select_by_department(department_list[12])

In [16]:
G = cl2.graph()
network_G, config_G = nw.visualize(G, plot_in_cell_below=False)
data_G = cl2.to_dict()

In [37]:
'02526' in cl2.courses

False

In [56]:
for c in cl2.catalogue['02601'].outs:
    print(c.course_id)
    print(c.course_id in cl2.courses)

12936
True


In [40]:
department_list

['01 Institut for Matematik og Computer Science',
 '10 Institut for Fysik',
 '12 Institut for Vand og Miljøteknologi',
 '22 Institut for Sundhedsteknologi',
 '23 Fødevareinstituttet',
 '25 Institut for Akvatiske Ressourcer',
 '26 Institut for Kemi',
 '27 Institut for Bioteknologi og Biomedicin',
 '28 Institut for Kemiteknik',
 '29 DTU Biosustain',
 '30 Institut for Rumforskning og -teknologi',
 '34 Institut for Fotonik',
 '41 Institut for Mekanisk Teknologi',
 '42 Institut for Teknologi, Ledelse og Økonomi',
 '46 Institut for Vindenergi',
 '47 Institut for Energikonvertering- og lagring',
 '56 DTU Nanolab',
 '62 Institut for Ingeniørteknologi og -didaktik',
 '88 Andre kurser']

In [42]:
levels_dict = defaultdict(lambda: [])
for course in cl.catalogue.values():
    levels_dict[course.level].append(course)

In [43]:
levels_dict

defaultdict(<function __main__.<lambda>()>,
            {0: [<__main__.Course at 0x22ca3f4f430>,
             1: [<__main__.Course at 0x22ca2f1cee0>,
             2: [<__main__.Course at 0x22ca2f1c940>,
             3: [<__main__.Course at 0x22ca2ea9580>,
             4: [<__main__.Course at 0x22ca2ea9760>,
             5: [<__main__.Course at 0x22ca2e5b0a0>,
             6: [<__main__.Course at 0x22ca2d1ff40>,
             7: [<__main__.Course at 0x22ca2d39ca0>,
             8: [<__main__.Course at 0x22ca2d39f40>,
             9: [<__main__.Course at 0x22ca441efa0>]})